# Intro

The `rrr` module provides convenience functions to record and replay a syzkaller reproducer (or any userspace stimulus) and trace it:

In [ ]:
import rrr
from rrr import Stimulus, Rootfs, Kernel

We want to create the files in the `out` directory so they are persistent outside of docker:

In [ ]:
import os
os.chdir("./out")

# Build

For example, given a syzbot bug ID, `rrr` can download and build:
- The kernel at the buggy commit and config
- A minimal rootfs that chains from busybox into the reproducer.

In [ ]:
with open('/root/repro.c', 'r') as f:
    repro_str = f.read()
stimulus = Stimulus.from_str(repro_str)
rootfs = Rootfs(stimulus)
kernel = Kernel('/root/kernel')

# Record

From there on, one can record the execution of the bug reproducer:

In [ ]:
record = rrr.record(kernel, rootfs)

# Trace

This record can be replayed at will, for example, under tracing:

In [ ]:
trace = rrr.trace(kernel, rootfs, record)

And this trace can be visualized in Perfetto

In [ ]:
rrr.serve_trace(trace)

## Replay

Once you've identified an interesting part of the trace from Perfetto, you can read the slice's start time in the "Raw" format and interpret it as an instruction number.



You can then replay the execution and attach gdb at that instruction number to go deeper with your debugging.

In [ ]:
rrr.replay(rootfs, record)